In [1]:
#coding: utf-8
import xml.etree.ElementTree as ET
import pickle
import os
from os import listdir, getcwd
from os.path import join
import glob
import json

import pdb

In [2]:
#-----类别信息-----#
classes = []
for i in range(1,19,1):
    classes.append(str(i))

#classes_ch = ["雀巢丝滑拿铁","饮料","泡面","红酒","零食","矿泉水","酸奶"]

In [5]:
#-----数据路径-----#

# 数据文件根目录
wd = "/mnt/lvmhdd/tanfulun/workspaces/Data/MissFreshSmartShelf_Exp"

# raw images(*.jpg) dir
raw_images_dir = os.path.join(wd,'JPEGImages')

# pascal-voc labels(*.xml) dir
voc_labels_dir = os.path.join(wd,"Annotations")

# imageset dir(train.txt/val.txt)
imageset_dir = os.path.join(wd,'ImageSets','Main')

In [4]:
'''划分训练集(training set)和验证集(validating set)'''

trainSet_list = []
valSet_list = []
testSet_list = []

val_train_ratio = [1,9] #验证集和训练集照片数量比例

## start
paths = glob.glob(os.path.join(raw_images_dir,'*.jpg'))
imgNum = len(paths)

tot = sum(val_train_ratio)
valNum = int(float(val_train_ratio[0])/tot*imgNum)
trainNum = imgNum-valNum

trainSet_list = paths[0:trainNum]
valSet_list = paths[trainNum:]

print("TrainNum:%d ,ValNum:%d"%(len(trainSet_list),len(valSet_list)))

#--save
trainf = open(os.path.join(imageset_dir,'train.txt'),'w')
for path in trainSet_list:
    trainf.write(path.split('/')[-1].strip()[0:-4]+'\n')
trainf.close()

valf = open(os.path.join(imageset_dir,'val.txt'),'w')
for path in valSet_list:
    valf.write(path.split('/')[-1].strip()[0:-4]+'\n')
valf.close()

TrainNum:296 ,ValNum:32


In [6]:
'''获取数据集绝对路径'''

dataset_dir = imageset_dir
image_dir = raw_images_dir
save_dir = wd

# --training set
train_set_file_path = os.path.join(dataset_dir,'train.txt')
f = open(train_set_file_path,'r')
name_list = f.readlines()
name_list = [x.strip() for x in name_list]
f.close()
#pdb.set_trace()
save_f = open(os.path.join(save_dir,'train.txt'),'w')
for name in name_list:
    save_f.write(image_dir+'/'+name+'.jpg'+'\n')
save_f.close()

# --val set
val_set_file_path = os.path.join(dataset_dir,'val.txt')
f = open(val_set_file_path,'r')
name_list = f.readlines()
name_list = [x.strip() for x in name_list]
f.close()
#pdb.set_trace()
save_f = open(os.path.join(save_dir,'val.txt'),'w')
for name in name_list:
    save_f.write(image_dir+'/'+name+'.jpg'+'\n')
save_f.close()

In [8]:
'''Pascal-Voc Labels to DK Labels'''

sets=[('#', 'train'),('#','val')]


def convert(size, box):
    '''
    [topleft_x,topleft_y,bottomdown_x,bottomdown_y]->[center_x,center_y,w,h]
    '''
    dw = 1./(size[0])
    dh = 1./(size[1])
    x = (box[0] + box[1])/2.0 - 1
    y = (box[2] + box[3])/2.0 - 1
    w = box[1] - box[0]
    h = box[3] - box[2]
    x = x*dw
    w = w*dw
    y = y*dh
    h = h*dh
    return (x,y,w,h)

def convert_annotation(workingdir,voc_xml_path, image_id):
    in_file = open(voc_xml_path)
    out_file = open('%s/dk_labels/%s.txt'%(workingdir, image_id), 'w')
    tree=ET.parse(in_file)
    root = tree.getroot()
    size = root.find('size')
    w = int(size.find('width').text)
    h = int(size.find('height').text)

    #pdb.set_trace()

    for obj in root.iter('object'):
        difficult = obj.find('difficult').text
        cls = obj.find('name').text
        
        #cls_utf = cls.encode('utf-8')
        #if cls_utf not in classes_ch or int(difficult)==1:
        #    continue
        #cls_id = classes_ch.index(cls_utf)
        
        if cls not in classes or int(difficult)==1:
            continue
        cls_id = classes.index(cls)
       
        print ("%s  %s %s"%(cls,image_id,cls_id))

        xmlbox = obj.find('bndbox')
        b = (float(xmlbox.find('xmin').text), float(xmlbox.find('xmax').text), float(xmlbox.find('ymin').text), float(xmlbox.find('ymax').text))
        bb = convert((w,h), b)
        out_file.write(str(cls_id) + " " + " ".join([str(a) for a in bb]) + '\n')

    in_file.close()
    out_file.close()

## dk format label save path
dk_label_abs_dir = os.path.join(wd,'dk_labels')
if not os.path.exists(dk_label_abs_dir):
    os.makedirs(dk_label_abs_dir)
    
## pascal-voc format label file(*.xml) dir
voc_xml_dir = voc_labels_dir
voc_xml_paths = glob.glob(os.path.join(voc_xml_dir,'*.xml'))

## raw images(*.jpg)
image_dir = raw_images_dir
image_paths = glob.glob(os.path.join(image_dir,'*.jpg'))

## 检查标注数据是否和图片数据是否一一对应
diff=False

jpg_names = []
xml_names = []
miss = []
for imgpath in image_paths:
    name = imgpath.split('/')[-1].strip()[0:-4]
    jpg_names.append(name)
for xml in voc_xml_paths:
    xml_name = xml.split('/')[-1].strip()[0:-4]
    xml_names.append(xml_name)

for i in jpg_names:
    if i not in xml_names:
        miss.append(i)
        diff = True
if diff:
    print("file missing!")
    print miss
    exit()
else:
    pass

#pdb.set_trace()

## 开始转换主程序
for _,dataset in sets:
    set_path = os.path.join(wd,'ImageSets/Main/%s.txt'%dataset)
    infile = open(set_path,'r')
    names = infile.read().strip().split('\n')
    infile.close()

    for name in names:
        if name in jpg_names:
            img_path = os.path.join(wd,'JPEGImages/%s.jpg'%name)
        else:
            print("file missing in JPEGImages")
            print name
            exit()
        voc_xml_path = os.path.join(wd,'Annotations/%s.xml'%name)
        convert_annotation(wd,voc_xml_path,name)

    #pdb.set_trace()


6  goods_0292 5
8  goods_0292 7
8  goods_0292 7
11  goods_0292 10
11  goods_0292 10
12  goods_0292 11
15  goods_0292 14
1  goods_0325 0
1  goods_0325 0
1  goods_0325 0
3  goods_0325 2
3  goods_0325 2
3  goods_0325 2
4  goods_0325 3
4  goods_0325 3
2  goods_0325 1
5  goods_0325 4
7  goods_0325 6
7  goods_0325 6
7  goods_0325 6
8  goods_0325 7
8  goods_0325 7
9  goods_0325 8
9  goods_0325 8
10  goods_0325 9
16  goods_0325 15
17  goods_0325 16
17  goods_0325 16
18  goods_0325 17
1  goods_0151 0
1  goods_0151 0
1  goods_0151 0
2  goods_0151 1
2  goods_0151 1
2  goods_0151 1
3  goods_0151 2
4  goods_0151 3
4  goods_0151 3
5  goods_0151 4
6  goods_0151 5
7  goods_0151 6
7  goods_0151 6
8  goods_0151 7
8  goods_0151 7
8  goods_0151 7
9  goods_0151 8
9  goods_0151 8
9  goods_0151 8
11  goods_0151 10
14  goods_0151 13
15  goods_0151 14
17  goods_0151 16
16  goods_0151 15
18  goods_0151 17
11  goods_0151 10
2  goods_0017 1
3  goods_0017 2
3  goods_0017 2
4  goods_0017 3
4  goods_0017 3
5  goods_

11  goods_0178 10
13  goods_0178 12
15  goods_0178 14
17  goods_0178 16
17  goods_0178 16
17  goods_0178 16
16  goods_0178 15
2  goods_0283 1
2  goods_0283 1
6  goods_0283 5
7  goods_0283 6
8  goods_0283 7
8  goods_0283 7
8  goods_0283 7
8  goods_0283 7
8  goods_0283 7
10  goods_0283 9
10  goods_0283 9
10  goods_0283 9
14  goods_0283 13
14  goods_0283 13
14  goods_0283 13
14  goods_0283 13
17  goods_0283 16
17  goods_0283 16
18  goods_0283 17
2  goods_0134 1
2  goods_0134 1
2  goods_0134 1
5  goods_0134 4
6  goods_0134 5
8  goods_0134 7
9  goods_0134 8
9  goods_0134 8
11  goods_0134 10
12  goods_0134 11
14  goods_0134 13
14  goods_0134 13
15  goods_0134 14
15  goods_0134 14
17  goods_0134 16
17  goods_0134 16
18  goods_0134 17
2  goods_0320 1
2  goods_0320 1
2  goods_0320 1
6  goods_0320 5
7  goods_0320 6
8  goods_0320 7
8  goods_0320 7
9  goods_0320 8
9  goods_0320 8
14  goods_0320 13
14  goods_0320 13
14  goods_0320 13
15  goods_0320 14
15  goods_0320 14
15  goods_0320 14
18  goods_0

3  goods_0258 2
6  goods_0258 5
6  goods_0258 5
7  goods_0258 6
8  goods_0258 7
8  goods_0258 7
8  goods_0258 7
8  goods_0258 7
10  goods_0258 9
11  goods_0258 10
11  goods_0258 10
14  goods_0258 13
14  goods_0258 13
14  goods_0258 13
15  goods_0258 14
15  goods_0258 14
2  goods_0030 1
3  goods_0030 2
5  goods_0030 4
6  goods_0030 5
10  goods_0030 9
10  goods_0030 9
11  goods_0030 10
14  goods_0030 13
14  goods_0030 13
17  goods_0030 16
18  goods_0030 17
18  goods_0030 17
1  goods_0113 0
5  goods_0113 4
6  goods_0113 5
8  goods_0113 7
11  goods_0113 10
11  goods_0113 10
18  goods_0113 17
2  goods_0141 1
3  goods_0141 2
5  goods_0141 4
5  goods_0141 4
6  goods_0141 5
6  goods_0141 5
7  goods_0141 6
7  goods_0141 6
8  goods_0141 7
10  goods_0141 9
12  goods_0141 11
12  goods_0141 11
14  goods_0141 13
14  goods_0141 13
14  goods_0141 13
15  goods_0141 14
17  goods_0141 16
18  goods_0141 17
18  goods_0141 17
18  goods_0141 17
10  goods_0141 9
4  goods_0307 3
4  goods_0307 3
2  goods_0307 1

1  goods_0193 0
3  goods_0193 2
4  goods_0193 3
5  goods_0193 4
6  goods_0193 5
7  goods_0193 6
7  goods_0193 6
8  goods_0193 7
8  goods_0193 7
8  goods_0193 7
9  goods_0193 8
10  goods_0193 9
11  goods_0193 10
11  goods_0193 10
11  goods_0193 10
14  goods_0193 13
18  goods_0193 17
4  goods_0322 3
2  goods_0322 1
2  goods_0322 1
2  goods_0322 1
5  goods_0322 4
6  goods_0322 5
8  goods_0322 7
8  goods_0322 7
10  goods_0322 9
10  goods_0322 9
10  goods_0322 9
14  goods_0322 13
14  goods_0322 13
14  goods_0322 13
14  goods_0322 13
15  goods_0322 14
15  goods_0322 14
17  goods_0322 16
18  goods_0322 17
18  goods_0322 17
8  goods_0269 7
8  goods_0269 7
8  goods_0269 7
11  goods_0269 10
11  goods_0269 10
12  goods_0269 11
12  goods_0269 11
12  goods_0269 11
2  goods_0044 1
3  goods_0044 2
4  goods_0044 3
5  goods_0044 4
5  goods_0044 4
6  goods_0044 5
7  goods_0044 6
7  goods_0044 6
8  goods_0044 7
8  goods_0044 7
8  goods_0044 7
10  goods_0044 9
10  goods_0044 9
11  goods_0044 10
14  goods_

2  goods_0183 1
2  goods_0183 1
2  goods_0183 1
3  goods_0183 2
3  goods_0183 2
4  goods_0183 3
5  goods_0183 4
7  goods_0183 6
8  goods_0183 7
9  goods_0183 8
9  goods_0183 8
11  goods_0183 10
12  goods_0183 11
14  goods_0183 13
15  goods_0183 14
18  goods_0183 17
6  goods_0265 5
6  goods_0265 5
6  goods_0265 5
6  goods_0265 5
8  goods_0265 7
11  goods_0265 10
11  goods_0265 10
15  goods_0265 14
15  goods_0265 14
16  goods_0265 15
1  goods_0083 0
2  goods_0083 1
2  goods_0083 1
3  goods_0083 2
3  goods_0083 2
4  goods_0083 3
4  goods_0083 3
5  goods_0083 4
6  goods_0083 5
7  goods_0083 6
7  goods_0083 6
8  goods_0083 7
8  goods_0083 7
8  goods_0083 7
9  goods_0083 8
9  goods_0083 8
13  goods_0083 12
6  goods_0083 5
17  goods_0083 16
17  goods_0083 16
11  goods_0297 10
11  goods_0297 10
11  goods_0297 10
12  goods_0297 11
13  goods_0297 12
1  goods_0003 0
2  goods_0003 1
2  goods_0003 1
3  goods_0003 2
3  goods_0003 2
4  goods_0003 3
4  goods_0003 3
5  goods_0003 4
6  goods_0003 5
7  g

1  goods_0313 0
1  goods_0313 0
1  goods_0313 0
1  goods_0313 0
3  goods_0313 2
3  goods_0313 2
3  goods_0313 2
4  goods_0313 3
4  goods_0313 3
4  goods_0313 3
5  goods_0313 4
7  goods_0313 6
7  goods_0313 6
7  goods_0313 6
8  goods_0313 7
8  goods_0313 7
9  goods_0313 8
9  goods_0313 8
10  goods_0313 9
17  goods_0313 16
17  goods_0313 16
17  goods_0313 16
2  goods_0182 1
2  goods_0182 1
2  goods_0182 1
3  goods_0182 2
4  goods_0182 3
6  goods_0182 5
7  goods_0182 6
8  goods_0182 7
8  goods_0182 7
9  goods_0182 8
9  goods_0182 8
9  goods_0182 8
12  goods_0182 11
14  goods_0182 13
15  goods_0182 14
17  goods_0182 16
17  goods_0182 16
18  goods_0182 17
1  goods_0176 0
1  goods_0176 0
1  goods_0176 0
3  goods_0176 2
4  goods_0176 3
6  goods_0176 5
8  goods_0176 7
8  goods_0176 7
9  goods_0176 8
9  goods_0176 8
11  goods_0176 10
13  goods_0176 12
15  goods_0176 14
16  goods_0176 15
17  goods_0176 16
1  goods_0216 0
3  goods_0216 2
5  goods_0216 4
6  goods_0216 5
6  goods_0216 5
6  goods_02

1  goods_0132 0
1  goods_0132 0
2  goods_0132 1
2  goods_0132 1
2  goods_0132 1
3  goods_0132 2
3  goods_0132 2
4  goods_0132 3
4  goods_0132 3
5  goods_0132 4
5  goods_0132 4
6  goods_0132 5
7  goods_0132 6
2  goods_0132 1
8  goods_0132 7
8  goods_0132 7
9  goods_0132 8
9  goods_0132 8
9  goods_0132 8
10  goods_0132 9
8  goods_0132 7
11  goods_0132 10
12  goods_0132 11
14  goods_0132 13
15  goods_0132 14
15  goods_0132 14
17  goods_0132 16
17  goods_0132 16
18  goods_0132 17
4  goods_0132 3
11  goods_0279 10
11  goods_0279 10
12  goods_0279 11
12  goods_0279 11
1  goods_0188 0
2  goods_0188 1
2  goods_0188 1
2  goods_0188 1
3  goods_0188 2
3  goods_0188 2
4  goods_0188 3
6  goods_0188 5
7  goods_0188 6
9  goods_0188 8
10  goods_0188 9
11  goods_0188 10
12  goods_0188 11
12  goods_0188 11
14  goods_0188 13
14  goods_0188 13
15  goods_0188 14
7  goods_0188 6
18  goods_0188 17
3  goods_0229 2
3  goods_0229 2
4  goods_0229 3
4  goods_0229 3
2  goods_0229 1
2  goods_0229 1
5  goods_0229 4


15  goods_0303 14
1  goods_0036 0
1  goods_0036 0
4  goods_0036 3
6  goods_0036 5
8  goods_0036 7
9  goods_0036 8
11  goods_0036 10
12  goods_0036 11
15  goods_0036 14
17  goods_0036 16
16  goods_0036 15
1  goods_0047 0
2  goods_0047 1
4  goods_0047 3
5  goods_0047 4
6  goods_0047 5
7  goods_0047 6
9  goods_0047 8
10  goods_0047 9
10  goods_0047 9
10  goods_0047 9
11  goods_0047 10
14  goods_0047 13
14  goods_0047 13
18  goods_0047 17
18  goods_0047 17
2  goods_0138 1
2  goods_0138 1
2  goods_0138 1
3  goods_0138 2
6  goods_0138 5
7  goods_0138 6
8  goods_0138 7
9  goods_0138 8
9  goods_0138 8
4  goods_0138 3
5  goods_0138 4
5  goods_0138 4
11  goods_0138 10
12  goods_0138 11
14  goods_0138 13
15  goods_0138 14
15  goods_0138 14
17  goods_0138 16
17  goods_0138 16
10  goods_0138 9
16  goods_0138 15
